In [1]:
"""
    Get cleaned data
"""

import numpy as np

dataset = np.load('./datasets/EEGMap/EEGMap5S.npy')
labels = np.load('./datasets/EEGMap/EEGMap5SLabels.npy')

In [2]:
print(dataset.shape)
print(labels.shape)

(3557, 24, 24, 3)
(3557,)


In [3]:
"""
    Split dataset
"""

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
shuf_dataset, shuf_labels = shuffle(dataset, labels)
X_train, X_test, y_train, y_test = train_test_split(shuf_dataset, shuf_labels, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)
print("Train set size: {}, Val set size: {}, Test set size: {}".format(len(X_train), len(X_val), len(X_test)))
print(y_train.shape)
print(X_train.shape)

Train set size: 2489, Val set size: 534, Test set size: 534
(2489,)
(2489, 24, 24, 3)


In [4]:
"""
    Model and train
"""

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

model_path = './models/'
json_name = 'CNN_EEGMap5S.json'
weight_name = 'CNN_EEGMap5S_tmp.h5'

batch_size = 16
epochs = 128
num_classes = 2

channels = X_train.shape[1]
img_height = X_train.shape[2]
img_width = X_train.shape[3]

print("Constructing MLP Model: ...", end='')
model = Sequential()
model.add(Conv2D(16, (2, 2), padding='same', input_shape= X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# MLP
model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
print(" Complete.")

print("Compiling MLP Model: ...", end='')
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print(" Complete.")

earlyStop = EarlyStopping(monitor='val_loss', patience=32, mode='min')
mcp_save = ModelCheckpoint(filepath=model_path+weight_name, save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')

print("Fitting MLP Model: ...")
#results = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=epochs, callbacks=[earlyStop, mcp_save, reduce_lr_loss], validation_data=(X_val, y_val))
#results = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=epochs, validation_data=(X_val, y_val))
results = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[mcp_save, reduce_lr_loss], validation_data=(X_val, y_val), shuffle=True)
#results = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), shuffle=True)

print("Complete.")

Using TensorFlow backend.


Constructing MLP Model: ... Complete.
Compiling MLP Model: ... Complete.
Fitting MLP Model: ...
Train on 2489 samples, validate on 534 samples
Epoch 1/128
2489/2489 [==============================] - 3s 1ms/step - loss: 0.6641 - acc: 0.6119 - val_loss: 0.6544 - val_acc: 0.6873
Epoch 2/128
2489/2489 [==============================] - 2s 613us/step - loss: 0.6351 - acc: 0.6842 - val_loss: 0.6094 - val_acc: 0.7097
Epoch 3/128
2489/2489 [==============================] - 2s 616us/step - loss: 0.6059 - acc: 0.7039 - val_loss: 0.5788 - val_acc: 0.7191
Epoch 4/128
2489/2489 [==============================] - 2s 616us/step - loss: 0.5624 - acc: 0.7240 - val_loss: 0.5233 - val_acc: 0.7678
Epoch 5/128
2489/2489 [==============================] - 2s 609us/step - loss: 0.5168 - acc: 0.7549 - val_loss: 0.5228 - val_acc: 0.7603
Epoch 6/128
2489/2489 [==============================] - 2s 627us/step - loss: 0.4852 - acc: 0.7742 - val_loss: 0.4826 - val_acc: 0.7509
Epoch 7/128
2489/2489 [==============

In [5]:
"""
    Save the model
"""

model_path = './models/'
json_name = 'CNN_EEGMap5S.json'
weight_name = 'CNN_EEGMap5S.h5'

model_json = model.to_json()
with open(model_path+json_name, 'w') as json_file:
    json_file.write(model_json)
#model.save_weights(model_path+weight_name)

In [6]:
"""
    Output & plot the results
    Aspect Ratio 2 to 1
"""

import matplotlib.pyplot as plt

image_path = './graphs/'
image_name = 'CNN_EEGMap5S_2to1.png'

fig = plt.figure(figsize=(10,5))
plt.plot(results.history['loss'], label='train loss')
plt.plot(results.history['val_loss'], label='val loss')
plt.plot(results.history['acc'], label='train acc')
plt.plot(results.history['val_acc'], label='val acc')
plt.ylim((0.2,1.5))
plt.legend(loc='lower left')
plt.title('AF7, AF8, TP9, TP10 EEG Maps 5S [1-50 Hz] CNN')
plt.xlabel('Epoch(s)')
plt.ylabel('Performance')
#plt.show()
plt.savefig(image_path+image_name, bbox_inches='tight')
test_scores = model.evaluate(X_test, y_test)
print("\nFinal test %s: %.2f%%" % (model.metrics_names[1], test_scores[1]*100))

534/534 [==============================] - 0s 162us/step

Final test acc: 90.26%


In [ ]:
"""
    Output & plot the results
    Aspect Ratio 2 to 1
"""

import matplotlib.pyplot as plt

image_path = './graphs/'
image_name = 'CNN_EEGMap5S_16to9.png'

fig = plt.figure(figsize=(16,9))
plt.plot(results.history['loss'], label='train loss')
plt.plot(results.history['val_loss'], label='val loss')
plt.plot(results.history['acc'], label='train acc')
plt.plot(results.history['val_acc'], label='val acc')
plt.legend(loc='lower left')
plt.title('AF7, AF8, TP9, TP10 EEG Maps 5S [1-50 Hz] CNN')
plt.xlabel('Epoch(s)')
plt.ylabel('Performance')
#plt.show()
plt.savefig(image_path+image_name, bbox_inches='tight')

In [ ]:
"""
    Try fetching the model
"""

model_path = './models/'
json_name = 'CNN_EEGMap5S.json'
weight_name = 'CNN_EEGMap5S.h5'

from keras.models import model_from_json

json_file = open(model_path+json_name)
loaded_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_json)
loaded_model.load_weights(model_path+weight_name)
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

loaded_test_score = loaded_model.evaluate(X_test, y_test)
print("\nFinal test %s: %.2f%%" % (loaded_model.metrics_names[1], loaded_test_score[1]*100))